In [5]:
import numpy as np
import pandas as pd
from scipy.stats import norm, poisson
from scipy.optimize import minimize
# import statsmodels.api as sm

# Synthetic data generation
def generate_data(n=1000, beta=[1, 0.5], censoring=True, dist="normal"):
    np.random.seed(42)
    X = np.random.normal(size=(n, len(beta)))
    if dist == "normal":
        y_star = X @ beta + np.random.normal(scale=1, size=n)
    elif dist == "poisson":
        lambda_ = np.exp(X @ beta)
        y_star = poisson.rvs(lambda_)
    y = np.maximum(0, y_star) if censoring else y_star
    return X, y

# Tobit MLE
def tobit_log_likelihood(params, X, y):
    beta, sigma = params[:-1], params[-1]
    y_star = X @ beta
    censored = y == 0
    ll_censored = np.log(norm.cdf(-y_star[censored] / sigma))
    ll_uncensored = norm.logpdf(y[~censored], loc=y_star[~censored], scale=sigma)
    return -(ll_censored.sum() + ll_uncensored.sum())

def fit_tobit(X, y):
    init_params = np.ones(X.shape[1] + 1)  # betas and sigma
    result = minimize(tobit_log_likelihood, init_params, args=(X, y), method="BFGS")
    return result.x

# Tobit-Poisson MLE
def tobit_poisson_log_likelihood(beta, X, y):
    lambda_ = np.exp(X @ beta)
    censored = y == 0
    ll_censored = -lambda_[censored]
    ll_uncensored = y[~censored] * np.log(lambda_[~censored]) - lambda_[~censored] - np.log(np.arange(1, y[~censored].max() + 1)).sum()
    return -(ll_censored.sum() + ll_uncensored.sum())

def fit_tobit_poisson(X, y):
    init_params = np.ones(X.shape[1])
    result = minimize(tobit_poisson_log_likelihood, init_params, args=(X, y), method="BFGS")
    return result.x

In [6]:
# Main workflow
X_normal, y_normal = generate_data(dist="normal")
X_poisson, y_poisson = generate_data(dist="poisson")

# Fit models
tobit_normal_params = fit_tobit(X_normal, y_normal)
tobit_poisson_params = fit_tobit_poisson(X_normal, y_normal)

poisson_poisson_params = fit_tobit_poisson(X_poisson, y_poisson)
poisson_normal_params = fit_tobit(X_poisson, y_poisson)

C:\Users\benja\AppData\Local\Temp\ipykernel_13552\4005871427.py:24: RuntimeWarning: divide by zero encountered in log
  ll_censored = np.log(norm.cdf(-y_star[censored] / sigma))
C:\Users\benja\AppData\Local\Temp\ipykernel_13552\4005871427.py:24: RuntimeWarning: divide by zero encountered in log
  ll_censored = np.log(norm.cdf(-y_star[censored] / sigma))


In [7]:
# Print results
print(f"Tobit Model Parameters (Normal Data):\t", tobit_normal_params)
print(f"Poisson Model Parameters (Normal Data):\t", poisson_normal_params)
print(f"Tobit Model Parameters (Poisson Data):\t", tobit_poisson_params)
print(f"Poisson Model Parameters (Poisson Data):", poisson_poisson_params)

Tobit Model Parameters (Normal Data):	 [1.01456973 0.52315274 0.97516277]
Poisson Model Parameters (Normal Data):	 [21003.16631965  6599.3593546  -3156.22361617]
Tobit Model Parameters (Poisson Data):	 [0.44614059 0.20731184]
Poisson Model Parameters (Poisson Data): [0.97071057 0.50767663]
